In [14]:
import cv2
# import matplotlib
# import matplotlib.pyplot as plt
import numpy as np
import glob
import torch
# import torch.nn as nn 
import time
import math
# import torch.nn.functional as F

from net import *
from utils import *
from tracker import *
import onnx


In [23]:
backbone_m = Inception3_M()
backbone_m.update_params()
backbone_q = Inception3_Q()
backbone_q.update_params()
neck_m = AdjustLayer()
neck_m.update_params()
neck_q = AdjustLayer()
neck_q.update_params()
head = STMHead()
head.update_params()

model = STMTrack(backbone_m, backbone_q, neck_m, neck_q, head)
# model.update_params()

# Convert BatchNorm to SyncBatchNorm 
# task_model = convert_model(task_model)
model_file = "new-epoch-19.pkl"

# model_file = "epoch-19.pkl"
model_state_dict = torch.load(model_file,
                        map_location=torch.device("cpu"))

# model.load_state_dict(model_state_dict['model_state_dict'])
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [24]:
net = model.basemodel_q

In [25]:
ONNX_FILE_PATH = "backbone_q.onnx"

In [28]:
input = torch.randn(1,3,289,289).cuda()
net.eval()
net.cuda()
torch.onnx.export(net, input, ONNX_FILE_PATH, input_names=["query_img"], output_names=["h_q"], export_params=True)

In [29]:
y = net(input)